##Example to sum select link flow results from all time periods to base scenario for export
####Setup and create attribute to store total select link flows
###Initial Script: Ziying Ouyang Feb 2019

##Setup to save results for multiple analyses
#####Same as the above steps, but re-arranged for multiple sequential calls, and optimized to only load the reference time period scenarios once.

In [2]:
import os
import pandas as pd
modeller = inro.modeller.Modeller()


#I/O
main_directory = os.path.dirname(os.path.dirname(modeller.desktop.project.path)).replace("\\","/")
import_path = os.path.join(main_directory, "emme_project\Database").replace("\\","/")
file_name_input = os.path.join(import_path, "Sl_query.csv")


suffixes = pd.read_csv(file_name_input,sep=',')

suffixes = suffixes['Query']

In [3]:
#Iterate through the suffix results and summarize the flows    
def summarize_select_link(suffix, base_scenario, network, ref_networks):
    total_name = "@sel_" + suffix
    if base_scenario.extra_attribute(total_name):
        base_scenario.delete_extra_attribute(total_name)
    flow_xatt = base_scenario.create_extra_attribute("LINK", total_name)
    flow_xatt.description = "Total select flow for %s" % suffix
    if total_name in network.attributes("LINK"):
        network.delete_attribute("LINK", total_name)
    network.create_attribute("LINK", total_name)      

    
    truck_classes = [
        ('TRKHGP', 2.5), ('TRKHTOLL', 2.5),
        ('TRKLGP', 1.3), ('TRKLTOLL', 1.3),
        ('TRKMGP', 1.5), ('TRKMTOLL', 1.5),
    ]
    
    auto_classes = [
        'SOVGPL', 'SOVTOLLL', 'HOV2HOVL', 'HOV2TOLLL', 'HOV3HOVL', 'HOV3TOLLL',
        'SOVGPM', 'SOVTOLLM', 'HOV2HOVM', 'HOV2TOLLM', 'HOV3HOVM', 'HOV3TOLLM',
        'SOVGPH', 'SOVTOLLH', 'HOV2HOVH', 'HOV2TOLLH', 'HOV3HOVH', 'HOV3TOLLH',
    ]

    
    auto_attrs = ["@sl_" + name.lower() + "_" + suffix
                  for name in auto_classes]
    truck_attrs = [("@sl_" + name.lower() + "_" + suffix, pce)
                   for name, pce in truck_classes]

    for ref_network in ref_networks:
        for link in base_network.links():
            ref_link = ref_network.link(link.i_node, link.j_node)
            flow = sum(ref_link[att] for att in auto_attrs)
            flow += sum(ref_link[att] / pce for att, pce in truck_attrs)
            link[total_name] += flow

            
            
modeller = inro.modeller.Modeller()
emmebank = modeller.emmebank
scenario_id = 100

base_scenario = emmebank.scenario(scenario_id)
base_network = base_scenario.get_network()
ref_networks = []
periods = ["EA", "AM", "MD", "PM", "EV"]


for number, period, in enumerate(periods, start=scenario_id + 1):
    scenario = emmebank.scenario(number)
    ref_networks.append(scenario.get_network())

#suffixes = ["dtwn","uptn","kmsa","svly"]
#suffixes = ["enci"]
for suffix in suffixes:
    summarize_select_link(suffix, base_scenario, base_network, ref_networks)

    
base_scenario.publish_network(base_network)

#network_to_shapefile = modeller.tool("inro.emme.data.network.export_network_as_shapefile")
#path = "..\\..\\output"
#report = network_to_shapefile(
#    export_path=path,
#    scenario=base_scenario,
#    selection = {"node": "all", "link": "all", "turn": "none", "transit_line": "none"})
    


ArgumentError: Invalid extra attribute ID: @sel_delm 